In [3]:
import time 
from datetime import datetime
Now = datetime.now()
Today = Now.strftime("%Y-%m-%d")
print("Today is:", Today)

Today is: 2025-07-31


In [12]:
from crewai.tools import tool

from langchain_community.tools import DuckDuckGoSearchResults

@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic."""
    return DuckDuckGoSearchResults().run(search_query)

import json
import yfinance as yf
from curl_cffi import requests
session= requests.Session(impersonate='chrome')

@tool("Get current stock price")
def get_current_stock_price(stock_symbol: str)-> str:
    """use this function to get the current stock price for a given stock symbol.
    Args: stock_symbol (str): The stock symbol.
    Returns: str: The current stock price or error message.
    """
    try:
        time.sleep(0.5)
        stock = yf.Ticker(stock_symbol,session=session)
        current_price = stock.info.get("regularMarketPrice",stock.info.get("currentPrice"))
        return f"The current price of {stock_symbol} is ${current_price:.2f}" if current_price else f"could not retrieve the current price for {stock_symbol}."
    except Exception as e:
        return f"Error fetching stock{stock_symbol} price: {str(e)}"
    
@tool
def get_company_info(stock_symbol: str) -> str:
    """use this function to get the company information and current financial snapshot for a given stock symbol.
    Args: stock_symbol (str): The stock symbol.
    Returns: json containing company profile and current financial snapshot .
    """
    try:
        company_info_full = yf.Ticker(stock_symbol,session=session).info
        if company_info_full is None:
            return f"No company information available for {stock_symbol}."
        company_info_cleaned = {
            "company_name": company_info_full.get("longName", "N/A"),
            "stock_symbol": company_info_full.get("symbol", stock_symbol),
            "current_price": company_info_full.get("regularMarketPrice", company_info_full.get("currentPrice", "N/A")),
            "market_cap": company_info_full.get("marketCap", "N/A"),
            "sector": company_info_full.get("sector", "N/A"),
            "industry": company_info_full.get("industry", "N/A"),
            "city": company_info_full.get("city", "N/A"),
            "country": company_info_full.get("country", "N/A"),
            "EPS": company_info_full.get("trailingEps", "N/A"),
            "PE_ratio": company_info_full.get("trailingPE", "N/A"),
            "website": company_info_full.get("website", "N/A"),
            "description": company_info_full.get("longBusinessSummary", "N/A")
        }
        return json.dumps(company_info_cleaned)
    except Exception as e:
        return f"Error fetching company information for {stock_symbol}: {str(e)}"
    
@tool
def get_income_statement(stock_symbol: str) -> str:
    """use this function to get the income statement for a given stock symbol.
    Args: stock_symbol (str): The stock symbol.
    Returns: json containing the income statement or an empty dictionary .
    """
    try:
        stock = yf.Ticker(stock_symbol,session=session)
        income_statement = stock.financials
        return income_statement.to_json(orient="index")
    except Exception as e:
        return f"Error fetching income statement for {stock_symbol}: {str(e)}"

In [29]:
from crewai import Agent
import os 
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

from crewai.llm import LLM
key = os.getenv("OPENAI_API_KEY")

llm = LLM(model="gpt-4o-mini", temperature=0.7)

news_info_explorer = Agent(
    role="News and info Information Researcher",
    goal="gather and provide information on current news and information and financial data about a company from the internet.",
    llm=llm,
    verbose=True,
    backstory=(
        "You are an expert researcher,who can gather detailed information about a company"
        'Consider you are on:'+Today
    ),
    tools=[search_tool],
    cache=True,
    max_iter=2,
)

data_explorer = Agent(
    role=" Data Researcher",
    goal="gather and provide financial data and company information about a stock   .",
    llm=llm,
    verbose=True,
    backstory=(
        "You are an expert data researcher, who can gather detailed data about a company or "
        "When using tools,use stock symbol and add a suffix '.NS' to it. try with and without suffix if it does not work."
        'Consider you are on:'+Today
    ),
    tools=[get_company_info, get_income_statement],
    cache=True,
    max_iter=2,
)

analyst = Agent(
    role="Data Analyst",
    goal="consolidate financial data ,stock information , and provide a summary",
    llm=llm,
    verbose=True,
    backstory=(
        "You are an expert investment analyst on financial data , stock/company related current information who can analyze detailed information about a company/stock "
        "making comprehensive analysis and providing insights.Use Indian units for numbers and currency(lakh ,crore)."
        'Consider you are on:'+Today
    )
)

fin_expert = Agent(
    role="Financial Expert",
    goal="considering financial analysis of a stock make investment decisions and recommendations",
    llm=llm,
    verbose=True,
    backstory=(
        "You are an expert financial advisor, who can investment recommendations"
        "Consider the financial analysis ,current information about the company,current stock and make investment decisions."
        "and make recommendations about whether to buy/hold/sell a stcok along with small reason."
        "When using tools,use stock symbol and add a suffix '.NS' to it. try with and without suffix if it does not work."
        'Consider you are on:'+Today
    ),
    tools=[get_current_stock_price],
    cache=True, 
    max_iter=2,
)

In [30]:
from crewai import Task

get_company_financial_info = Task(
    description="Get company financial information like income statement, company profile, and other fundamental ratios ",
    expected_output="Detailed information from income statement, key ratios for {stock}. "
                    "Indicate also about current financial status and trend over the period.",
    agent=data_explorer,
)

get_company_news=Task(
    description="Get current news and information about a company:{stock}.",
    expected_output="Current news and information and business data about {stock}.Provide a summary of the news and information.",
    agent=news_info_explorer,
)

analyse= Task(
    description="Make through analysis based on given financial data and latest news of a stock ",
    expected_output="Detailed analysis of the financial data and current information about {stock}.",
    agent=analyst,
    context=[get_company_financial_info, get_company_news],
    output_file="Analysis.md"
)

advise= Task(
    description="Make a recommendation about the investment in a stock , based on analysis provided."
                "Explain the reasons.",
    expected_output="Recommendation (Buy/Hold/Sell) of a stock backed with reasons elaborated. "
                "Response in Mark down format.",  

    agent=fin_expert,
    context=[analyse],
    output_file="Investment_recommendation.md"
)

In [31]:
from crewai import Crew,Process
from datetime import datetime

def time_stamp(Input):
    print(datetime.now())

crew=Crew(
    agents=[news_info_explorer, data_explorer, analyst, fin_expert],
    tasks=[get_company_financial_info, get_company_news, analyse, advise],
    verbose=True,
    Process=Process.sequential,
    step_callback=time_stamp,
)
    

In [32]:
result =crew.kickoff(inputs={"stock": "RELIANCE"})
print("Final Result:", result)

┌────────────────────────── Crew Execution Started ───────────────────────────┐
│                                                                             │
│  Crew Execution Started                                                     │
│  Name: crew                                                                 │
│  ID: f74e8114-a6b3-48ab-a72b-16697683007a                                   │
│  Tool Args:                                                                 │
│                                                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

🚀 Crew: crew
└── 📋 Task: bd1ce8f5-5076-4537-9dda-e082f551ea9d
    Status: Executing Task...
┌───────────────────────────── 🤖 Agent Started ──────────────────────────────┐
│                                                                             │
│  Agent:  Data Researcher  